In [26]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from sklearn.cluster import KMeans

In [27]:
page = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

soup = BeautifulSoup(page.text, 'html.parser')

In [28]:
table_contents=[]
table = soup.find('table')

for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)
        
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

In [29]:
df

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto Business,Enclave of M4L
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


### FOllowing code not working , so I have downaloed file from internet to get Coordinates
```python
import geocoder

def getCoordinates(postal_code):
    lat_lng_coords = None
    count = 0
    while(lat_lng_coords is None):
        print("count:",count," start")
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    return lat_lng_coords[0], lat_lng_coords[1]
```

I have download the files from following link
[Candian Coordinates](https://www.serviceobjects.com/blog/free-zip-code-and-postal-code-database-with-geocoordinates/)

In [30]:
coo = pd.read_csv('/Users/ayyaz/Downloads/ZipCodeFiles/CanadianPostalCodes202103.csv')

In [31]:
# Review the results
coo

,POSTAL_CODE,CITY,PROVINCE_ABBR,TIME_ZONE,LATITUDE,LONGITUDE
0,A0A 1Z0,CONCEPTION HARBOUR,NL,3,47.427521,-53.225612
1,A0A 2B0,CUPIDS,NL,3,47.526681,-53.259703
2,A0A 3H0,PETTY HARBOUR,NL,3,47.476427,-52.707683
3,A0A 3W0,SOUTH RIVER,NL,3,47.535964,-53.209132
4,A0A 4J0,WESTERN BAY,NL,3,47.883773,-53.084634
...,...,...,...,...,...,...
889667,Y1A 6P3,WHITEHORSE,YT,8,60.720622,-135.069191
889668,Y1A 6R8,WHITEHORSE,YT,8,60.719984,-135.053639
889669,Y1A 6S3,WHITEHORSE,YT,8,60.719986,-135.053639
889670,Y1A 6S9,WHITEHORSE,YT,8,60.700351,-135.109574


In [32]:
# dropping useless columns 
coo.drop(['CITY','PROVINCE_ABBR','TIME_ZONE'], axis=1, inplace=True )

In [33]:
coo

,POSTAL_CODE,LATITUDE,LONGITUDE
0,A0A 1Z0,47.427521,-53.225612
1,A0A 2B0,47.526681,-53.259703
2,A0A 3H0,47.476427,-52.707683
3,A0A 3W0,47.535964,-53.209132
4,A0A 4J0,47.883773,-53.084634
...,...,...,...
889667,Y1A 6P3,60.720622,-135.069191
889668,Y1A 6R8,60.719984,-135.053639
889669,Y1A 6S3,60.719986,-135.053639
889670,Y1A 6S9,60.700351,-135.109574


In [34]:
# Keeping only data related to Toronto with postal code starts with M 
coo = coo[coo['POSTAL_CODE'].str.startswith('M', na=True)]

In [35]:
#we need only the first part of the postal code, so adding a column after spliting from POSTAL_CODE column value
coo['PostalCode'] = coo.apply(lambda row: row.POSTAL_CODE.split()[0], axis=1)

<ipython-input-35-aa308c2fa7ce>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  coo['PostalCode'] = coo.apply(lambda row: row.POSTAL_CODE.split()[0], axis=1)


In [36]:
coo.reset_index(inplace=True)
coo

,index,POSTAL_CODE,LATITUDE,LONGITUDE,PostalCode
0,345260,M1B 0B7,43.806686,-79.194353,M1B
1,345261,M1B 1G2,43.802762,-79.154572,M1B
2,345262,M1B 1H2,43.804741,-79.158141,M1B
3,345263,M1B 1N4,43.801117,-79.210678,M1B
4,345264,M1B 1N9,43.802552,-79.211619,M1B
...,...,...,...,...,...
49971,846655,M9W 6X9,43.703223,-79.589554,M9W
49972,846656,M9W 6Z4,43.723126,-79.600233,M9W
49973,846657,M9W 7A1,43.672359,-79.593122,M9W
49974,846658,M9W 7B5,43.729609,-79.624876,M9W


As we have multiple areas in one postal code, so Mean() will be the good choice to get LATITUDE & LONGITUE againts one Postal code


In [37]:
coo = coo.groupby('PostalCode').mean()

In [38]:
coo.reset_index(inplace=True)
coo

,PostalCode,index,LATITUDE,LONGITUDE
0,M1B,494681.876380,43.806001,-79.206321
1,M1C,484067.885897,43.804861,-79.058215
2,M1E,486523.624204,43.765596,-79.190358
3,M1G,491197.702544,43.768293,-79.217310
4,M1H,482294.110000,43.766641,-79.238566
...,...,...,...,...
98,M9N,527922.235174,43.706586,-79.516584
99,M9P,506224.496994,43.692928,-79.530811
100,M9R,524534.164049,43.688227,-79.552526
101,M9V,525305.183136,43.742565,-79.584813


##### Now we will join 'coo' and 'df'

In [39]:
df = pd.merge(coo, df, on="PostalCode")

In [40]:
df

,PostalCode,index,LATITUDE,LONGITUDE,Borough,Neighborhood
0,M1B,494681.876380,43.806001,-79.206321,Scarborough,"Malvern, Rouge"
1,M1C,484067.885897,43.804861,-79.058215,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,486523.624204,43.765596,-79.190358,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,491197.702544,43.768293,-79.217310,Scarborough,Woburn
4,M1H,482294.110000,43.766641,-79.238566,Scarborough,Cedarbrae
...,...,...,...,...,...,...
98,M9N,527922.235174,43.706586,-79.516584,York,Weston
99,M9P,506224.496994,43.692928,-79.530811,Etobicoke,Westmount
100,M9R,524534.164049,43.688227,-79.552526,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ..."
101,M9V,525305.183136,43.742565,-79.584813,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."


In [41]:
df.groupby(['Borough']).count()

,PostalCode,index,LATITUDE,LONGITUDE,Neighborhood
Borough,,,,,
Central Toronto,9,9,9,9,9
Downtown Toronto,17,17,17,17,17
Downtown Toronto Stn A,1,1,1,1,1
East Toronto,4,4,4,4,4
East Toronto Business,1,1,1,1,1
East York,4,4,4,4,4
East York/East Toronto,1,1,1,1,1
Etobicoke,11,11,11,11,11
Etobicoke Northwest,1,1,1,1,1


### Only boroughs that contain the word Toronto (as per assignment)

In [42]:
neighborhoods = df[df['Borough'].str.contains('Toronto')]

In [43]:
neighborhoods.groupby(['Borough']).count()

,PostalCode,index,LATITUDE,LONGITUDE,Neighborhood
Borough,,,,,
Central Toronto,9,9,9,9,9
Downtown Toronto,17,17,17,17,17
Downtown Toronto Stn A,1,1,1,1,1
East Toronto,4,4,4,4,4
East Toronto Business,1,1,1,1,1
East York/East Toronto,1,1,1,1,1
West Toronto,6,6,6,6,6


In [44]:
from geopy.geocoders import Nominatim 
import folium

In [45]:
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of {} City are {}, {}.'.format(address,latitude, longitude))

The geograpical coordinate of Toronto City are 43.6534817, -79.3839347.


In [46]:
_map = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, borough, neighborhood in zip(neighborhoods['LATITUDE'], neighborhoods['LONGITUDE'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(borough+":", neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(_map)  
    
_map

In [47]:
neighborhoods

,PostalCode,index,LATITUDE,LONGITUDE,Borough,Neighborhood
37,M4E,507067.706320,43.678178,-79.295097,East Toronto,The Beaches
40,M4J,495629.979508,43.685824,-79.336850,East York/East Toronto,The Danforth East
41,M4K,488561.068266,43.680048,-79.351302,East Toronto,"The Danforth West, Riverdale"
42,M4L,506662.142599,43.670699,-79.317267,East Toronto,"India Bazaar, The Beaches West"
43,M4M,494260.916667,43.662278,-79.344262,East Toronto,Studio District
44,M4N,499611.980603,43.726629,-79.394431,Central Toronto,Lawrence Park
45,M4P,511962.649351,43.710818,-79.390425,Central Toronto,Davisville North
46,M4R,499140.373333,43.718157,-79.413008,Central Toronto,North Toronto West
47,M4S,495310.931319,43.702891,-79.387692,Central Toronto,Davisville
48,M4T,513339.815498,43.688746,-79.387443,Central Toronto,"Moore Park, Summerhill East"


In [48]:
CLIENT_ID = 'DALRYGD3YYDWGCNBT5SV2T2GAWAH4QOAJQIZ4NXAXH12KVEI' 
CLIENT_SECRET = '34ZVMV0NBGOL2NE1LHVJXQB3O5U3R34SYQZ3PKZ5L1QT5XWP' 
VERSION = '20180605' 
LIMIT = 100

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: DALRYGD3YYDWGCNBT5SV2T2GAWAH4QOAJQIZ4NXAXH12KVEI
CLIENT_SECRET:34ZVMV0NBGOL2NE1LHVJXQB3O5U3R34SYQZ3PKZ5L1QT5XWP


In [49]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [51]:
_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                          latitudes=neighborhoods['LATITUDE'],
                          longitudes=neighborhoods['LONGITUDE']
                         )

The Beaches
The Danforth  East
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North & West
The Annex, North Midtown, Yorkville
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Enclave of M5E
First Canadian Place, Underground city
Christie
Dufferin, Dovercourt Village
Little Portugal, Trinity
Brockton, Parkdale Village, Exhibition Place
High Park, The Junction 

In [53]:
_venues

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.678178,-79.295097,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.678178,-79.295097,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.678178,-79.295097,Beaches Bake Shop,43.680363,-79.289692,Bakery
3,The Beaches,43.678178,-79.295097,Grover Pub and Grub,43.679181,-79.297215,Pub
4,The Beaches,43.678178,-79.295097,Upper Beaches,43.680563,-79.292869,Neighborhood
...,...,...,...,...,...,...,...
1690,Enclave of M4L,43.664916,-79.313397,Balmy Beach Canoe Club,43.661104,-79.313564,Harbor / Marina
1691,Enclave of M4L,43.664916,-79.313397,Orchard Park,43.668858,-79.314160,Park
1692,Enclave of M4L,43.664916,-79.313397,Pizzaiolo,43.668953,-79.311683,Pizza Place
1693,Enclave of M4L,43.664916,-79.313397,Jody's Deli,43.668946,-79.311635,Sandwich Place


In [54]:
_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,100,100,100,100,100,100
"Brockton, Parkdale Village, Exhibition Place",66,66,66,66,66,66
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",64,64,64,64,64,64
Central Bay Street,60,60,60,60,60,60
Christie,9,9,9,9,9,9
Church and Wellesley,72,72,72,72,72,72
"Commerce Court, Victoria Hotel",81,81,81,81,81,81
Davisville,28,28,28,28,28,28
Davisville North,20,20,20,20,20,20


In [55]:
print('There are {} uniques categories.'.format(len(_venues['Venue Category'].unique())))

There are 239 uniques categories.


In [56]:
_onehot = pd.get_dummies(_venues[['Venue Category']], prefix="", prefix_sep="")

_onehot['Neighborhood'] = _venues['Neighborhood'] 

fixed_columns = [_onehot.columns[-1]] + list(_onehot.columns[:-1])
_onehot = _onehot[fixed_columns]

_onehot.head()

,Yoga Studio,Accessories Store,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,...,Toy / Game Store,Trail,Train Station,Udon Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar
0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [57]:
_grouped = _onehot.groupby('Neighborhood').mean().reset_index()
_grouped

,Neighborhood,Yoga Studio,Accessories Store,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,...,Toy / Game Store,Trail,Train Station,Udon Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar
0,Berczy Park,0.010000,0.000000,0.000000,0.01,0.010000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.015152,0.000000,0.00,0.045455,0.000000,0.030303,0.000000,0.015152,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.015152,0.000000,0.000000,0.000000,0.000000
2,"CN Tower, King and Spadina, Railway Lands, Har...",0.031250,0.000000,0.000000,0.00,0.015625,0.000000,0.000000,0.015625,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.015625,0.000000,0.000000,0.000000,0.000000
3,Central Bay Street,0.000000,0.000000,0.000000,0.00,0.000000,0.016667,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.016667,0.000000,0.000000,0.000000,0.016667
4,Christie,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,Church and Wellesley,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,"Commerce Court, Victoria Hotel",0.012346,0.000000,0.000000,0.00,0.000000,0.000000,0.012346,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.012346,0.000000,0.012346,0.000000,0.000000
7,Davisville,0.000000,0.000000,0.035714,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.035714,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,Davisville North,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,"Dufferin, Dovercourt Village",0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [58]:
_grouped.shape

(39, 239)

In [59]:
num_top_venues = 5

for hood in _grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = _grouped[_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                 venue  freq
0          Coffee Shop  0.08
1                 Café  0.05
2           Restaurant  0.04
3  Japanese Restaurant  0.04
4   Italian Restaurant  0.04


----Brockton, Parkdale Village, Exhibition Place----
         venue  freq
0          Bar  0.08
1         Café  0.06
2   Restaurant  0.06
3  Art Gallery  0.05
4    Gift Shop  0.05


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                venue  freq
0         Coffee Shop  0.06
1                Café  0.06
2  Italian Restaurant  0.05
3          Restaurant  0.05
4      Sandwich Place  0.05


----Central Bay Street----
                 venue  freq
0          Coffee Shop  0.18
1  Japanese Restaurant  0.05
2      Bubble Tea Shop  0.03
3   Italian Restaurant  0.03
4                 Café  0.03


----Christie----
           venue  freq
0           Café  0.33
1  Grocery Store  0.22
2     Baby Store  0.11
3    Coffee Shop  0.11
4  

4  Italian Restaurant  0.04




In [60]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [61]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = _grouped['Neighborhood']

for ind in np.arange(_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Café,Hotel,Bakery,Italian Restaurant,Japanese Restaurant,Restaurant,Cocktail Bar,Seafood Restaurant,Gastropub
1,"Brockton, Parkdale Village, Exhibition Place",Bar,Café,Restaurant,Art Gallery,Gift Shop,Coffee Shop,Lounge,Park,Arts & Crafts Store,Pizza Place
2,"CN Tower, King and Spadina, Railway Lands, Har...",Coffee Shop,Café,Restaurant,Italian Restaurant,Sandwich Place,Gym,Bar,Spa,Beer Bar,French Restaurant
3,Central Bay Street,Coffee Shop,Japanese Restaurant,Clothing Store,Café,Sandwich Place,Bubble Tea Shop,Italian Restaurant,Bike Rental / Bike Share,Discount Store,Bookstore
4,Christie,Café,Grocery Store,Park,Candy Store,Coffee Shop,Baby Store,Wine Bar,Electronics Store,Dumpling Restaurant,Eastern European Restaurant


In [72]:
neighborhoods_venues = neighborhoods_venues_sorted.copy()

In [73]:
_clustering = _grouped.drop('Neighborhood', 1)

_clustering

,Yoga Studio,Accessories Store,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,...,Toy / Game Store,Trail,Train Station,Udon Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar
0,0.010000,0.000000,0.000000,0.01,0.010000,0.000000,0.000000,0.000000,0.000000,0.010000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.015152,0.000000,0.00,0.045455,0.000000,0.030303,0.000000,0.015152,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.015152,0.000000,0.000000,0.000000,0.000000
2,0.031250,0.000000,0.000000,0.00,0.015625,0.000000,0.000000,0.015625,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.015625,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.00,0.000000,0.016667,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.016667,0.000000,0.000000,0.000000,0.016667
4,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.012346,0.000000,0.000000,0.00,0.000000,0.000000,0.012346,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.012346,0.000000,0.012346,0.000000,0.000000
7,0.000000,0.000000,0.035714,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.035714,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [74]:
kmeans = KMeans(n_clusters=3, random_state=3).fit(_clustering)

kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

 dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [75]:
neighborhoods_venues.insert(0, 'Cluster Labels', kmeans.labels_)
merged = neighborhoods

merged = merged.join(neighborhoods_venues.set_index('Neighborhood'), on='Neighborhood')

merged

,PostalCode,index,LATITUDE,LONGITUDE,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,M4E,507067.706320,43.678178,-79.295097,East Toronto,The Beaches,1,Church,Health Food Store,Bakery,Trail,Pub,Electronics Store,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant
40,M4J,495629.979508,43.685824,-79.336850,East York/East Toronto,The Danforth East,1,Convenience Store,Coffee Shop,Intersection,Park,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space
41,M4K,488561.068266,43.680048,-79.351302,East Toronto,"The Danforth West, Riverdale",1,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Furniture / Home Store,Café,Restaurant,Bookstore,Pizza Place,Bubble Tea Shop
42,M4L,506662.142599,43.670699,-79.317267,East Toronto,"India Bazaar, The Beaches West",1,Indian Restaurant,Grocery Store,Sandwich Place,Restaurant,Pizza Place,Fast Food Restaurant,Diner,Brewery,Café,Park
43,M4M,494260.916667,43.662278,-79.344262,East Toronto,Studio District,1,Bar,Coffee Shop,Bakery,Café,American Restaurant,Italian Restaurant,Park,Sandwich Place,Cheese Shop,Chinese Restaurant
44,M4N,499611.980603,43.726629,-79.394431,Central Toronto,Lawrence Park,1,Pharmacy,Pool,Park,Dim Sum Restaurant,Dive Bar,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
45,M4P,511962.649351,43.710818,-79.390425,Central Toronto,Davisville North,1,Pizza Place,Gym,Indian Restaurant,Pharmacy,Gym / Fitness Center,Deli / Bodega,Department Store,Dessert Shop,Sandwich Place,Middle Eastern Restaurant
46,M4R,499140.373333,43.718157,-79.413008,Central Toronto,North Toronto West,1,Skating Rink,Photography Studio,Playground,Discount Store,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
47,M4S,495310.931319,43.702891,-79.387692,Central Toronto,Davisville,1,Coffee Shop,Dessert Shop,Sandwich Place,Café,Italian Restaurant,Pizza Place,Gym,Farmers Market,Fast Food Restaurant,Seafood Restaurant
48,M4T,513339.815498,43.688746,-79.387443,Central Toronto,"Moore Park, Summerhill East",1,Café,Park,Thai Restaurant,Deli / Bodega,Gym / Fitness Center,Gym,Grocery Store,Japanese Restaurant,Sandwich Place,Doner Restaurant


In [76]:
merged.groupby('Cluster Labels').count()

,PostalCode,index,LATITUDE,LONGITUDE,Borough,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
Cluster Labels,,,,,,,,,,,,,,,,
0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
1,37,37,37,37,37,37,37,37,37,37,37,37,37,37,37,37
2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
